# **Import Modules**

In [5]:
import sys

import os

import pickle

sys.path.insert(0, '../input/deeplearning-utils')

In [6]:
from utils.ops import plot

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

from tqdm import tqdm

In [8]:
from cuml import TSNE
from cuml import SVC, KNeighborsClassifier, RandomForestClassifier

# **Load Data**

In [9]:
src_dir = '../input/ci-sc22-places-and-scene-recognition/'

In [15]:
x_train, y_train = dict(np.load('../input/cisc22placesandscenefeatures/train_data.npz')).values()
x_val, y_val = dict(np.load('../input/cisc22placesandscenefeatures/val_data.npz')).values()
x_test, test_ids = dict(np.load('../input/cisc22placesandscenefeatures/test_data.npz')).values()

y_train = y_train.argmax(axis=-1)
y_val = y_val.argmax(axis=-1)

In [17]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')

x_test = x_test.astype('float32')

# **Boosting Head**

In [93]:
# clf = RandomForestClassifier(n_estimators=100, max_depth=8)

clf = SVC(kernel='rbf', C=0.75, probability=True)

# clf = KNeighborsClassifier(n_neighbors=3)

clf.fit(x_train, y_train)

SVC()

In [94]:
y_pred = clf.predict(x_train)

print(accuracy_score(y_train, y_pred))

y_pred = clf.predict(x_val)

print(accuracy_score(y_val, y_pred))

0.9806719947806231
0.9223443223443224


# **Submission**

In [ ]:
p_train = clf.predict_proba(x_train)
p_val = clf.predict_proba(x_val)
p_test = clf.predict_proba(x_test)

np.savez('train_svc_0.npz', y=p_train)
np.savez('val_svc_0.npz', y=p_val)
np.savez('test_svc_0.npz', y=p_test)

In [101]:
y_test = clf.predict(x_test)

In [102]:
submission = pd.read_csv(src_dir + 'sample_submission.csv')

submission.drop([0, 1], axis=0, inplace=True)

submission['image_name'] = test_ids
submission['label'] = y_test

In [103]:
submission.to_csv('submission.csv', index=False, encoding='utf-8')

In [104]:
submission.head(5)

,image_name,label
0,22735.jpg,0
1,22706.jpg,0
2,13288.jpg,2
3,3863.jpg,1
4,22608.jpg,3
